In [6]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader


CATSnDOGS = np.array(pd.read_csv("CATSnDOGS.csv"))
Labels = np.array(pd.read_csv("Labels.csv"))

CATSnDOGS = CATSnDOGS.reshape(-1, 1, 64, 64)
data = torch.from_numpy(CATSnDOGS).float()
labels = torch.from_numpy(Labels.ravel()).long()

dataset = TensorDataset(data, labels)


# Split data into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [11]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=32*16*16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 32*16*16)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

def train(model, train_loader, val_loader, num_epochs, lr):
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model
    for epoch in range(num_epochs):
        # Set model to training mode
        model.train()

        # Iterate over batches in the training data
        for inputs, labels in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization step
            loss.backward()
            optimizer.step()

        # Evaluate the model on the validation set
        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_acc += torch.sum(preds == labels.data)

        # Print training and validation metrics
        train_loss = loss.item()
        train_acc = torch.sum(preds == labels.data)
        print("Epoch {}/{} - Training Loss: {:.4f} - Training Accuracy: {:.4f} - Validation Loss: {:.4f} - Validation Accuracy: {:.4f}".format(epoch+1, num_epochs, train_loss, train_acc/len(train_loader.dataset), val_loss/len(val_loader.dataset), val_acc/len(val_loader.dataset)))

    # Return the trained model
    return model


In [14]:
model = CNN()
num_epochs = 50
lr = 0.001

cnn = train(model, train_loader, val_loader, num_epochs, lr)

Epoch 1/50 - Training Loss: 19.9204 - Training Accuracy: 0.0190 - Validation Loss: 18.0182 - Validation Accuracy: 0.5000
Epoch 2/50 - Training Loss: 3.9516 - Training Accuracy: 0.0316 - Validation Loss: 5.3018 - Validation Accuracy: 0.5000
Epoch 3/50 - Training Loss: 2.8805 - Training Accuracy: 0.0190 - Validation Loss: 2.6047 - Validation Accuracy: 0.5250
Epoch 4/50 - Training Loss: 0.4019 - Training Accuracy: 0.0380 - Validation Loss: 0.4001 - Validation Accuracy: 0.8000
Epoch 5/50 - Training Loss: 0.1195 - Training Accuracy: 0.0506 - Validation Loss: 0.2774 - Validation Accuracy: 0.9000
Epoch 6/50 - Training Loss: 0.1661 - Training Accuracy: 0.0506 - Validation Loss: 0.2928 - Validation Accuracy: 0.8500
Epoch 7/50 - Training Loss: 0.1448 - Training Accuracy: 0.0506 - Validation Loss: 0.2697 - Validation Accuracy: 0.8750
Epoch 8/50 - Training Loss: 0.0812 - Training Accuracy: 0.0506 - Validation Loss: 0.2352 - Validation Accuracy: 0.9250
Epoch 9/50 - Training Loss: 0.0622 - Training 